This notebook is a boilerplate notebooks with widgets for:
 - uploading a geojson/ shpaefile
 - downloading a processed artefact - raster
 - downloading a processed artefact - vector

In [1]:
import ipywidgets as widgets
from IPython.display import display, HTML
from ipyleaflet import Map, WidgetControl, GeoData
import geopandas as gpd
import io
import base64

import rioxarray
import xarray as xr
import xarray_leaflet
import matplotlib.pyplot as plt

In [2]:
#import test geojson
geom = gpd.read_file('evergreen-resources/data/temp/dissolved-boundaries.geojson')
geom_geodata = GeoData(geo_dataframe = geom)
geom.head()


,fid,geometry
0,1,"POLYGON ((116.26012 -29.22530, 116.26172 -29.2..."


In [3]:

#this will be replaced by the actual data collected for the AOI
test_download = gpd.read_file('evergreen-resources/data/temp/points_subset.geojson')

In [4]:
test_raster = rioxarray.open_rasterio('evergreen-resources/data/temp/clipped_raster.tif')
test_raster = test_raster.rio.reproject('EPSG:4326')
test_raster = test_raster.sel(band=1)
#test_raster.plot()

RasterioIOError: evergreen-resources/data/temp/clipped_raster.tif: No such file or directory

In [ ]:
style = {'description_width': 'initial'}
uploader = widgets.FileUpload(
    description='Upload file',
    accept='.geojson',
    multiple=False,
    button_style='primary',
    style=style,
)

submit = widgets.Button(description='Display AOI', button_style='success', tooltip='Click me', style=style)

reset = widgets.Button(description='Reset', button_style='warning', tooltip='Click me', style=style)

get_data = widgets.Button(description='Get data for AOI', button_style='primary', tooltip='Click me', style=style)

In [ ]:
Map = Map(height='500px', width='500px', center=(-29.2253, 116.26012), zoom=10) #note that the center coordinates are the opposite to what is stored in a geojson, you need to swap the lat/long

In [ ]:
output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

In [ ]:
with output_widget:
    display(uploader)
    display(submit)
    display(get_data)
    display(reset)

In [ ]:

geo_json = None

def get_vector(upload_widget):
    vector = None
    try:
        file = upload_widget.value[0]
        content = file['content']
        bytes_io = io.BytesIO(content)
        vector = gpd.read_file(bytes_io)

    except Exception as e:
        print(e)
    return vector


def submit_clicked(b):
    global geo_json
    try:
        fc = get_vector(uploader)
        geo_json = GeoData(geo_dataframe = fc)
        Map.add_layer(geo_json)
    except Exception as e:
        print(e)
    
submit.on_click(submit_clicked)

In [ ]:

def reset_clicked(b):
    global geo_json  # Declare geo_json as global so that we can access it
    output_widget.clear_output()
    with output_widget:
        display(uploader)
        display(submit)
        display(get_data)
        display(reset)
    uploader.value.clear()
    uploader._counter = 0
    if geo_json is not None:  # If a GeoData layer has been added to the map
        Map.remove_layer(geo_json)  # Remove the GeoData layer from the map
        geo_json = None  # Reset the reference to the GeoData layer

reset.on_click(reset_clicked)

def create_download_link(df, title = "Download GeoDataFrame", filename = "data.geojson"): 
    df.to_file(filename, driver='GeoJSON')
    data = open(filename, "rb").read()
    b64 = base64.b64encode(data)
    payload = b64.decode()

    html = '<a download="{filename}" href="data:text/json;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
download_button = create_download_link(test_download)

In [ ]:
Map

In [ ]:
l = test_raster.leaflet.plot(Map, colormap=plt.cm.terrain)

In [ ]:
uploader.value

In [ ]:
display(download_button)